# Markov models: What could go wrong?
<img alt="FU BERLIN LOGO" style="border-width:0" src="https://www.fu-berlin.de/assets/default2/fu-logo-1x-2e301182ca3c9a45ae34adf6dca8b6ba.png" title='FU BERLIN.' align="right"/>

```
# Tim Hempel
# Markov Winterschool 2022
# Noe Group, FU Berlin
```

In [ ]:
import numpy as np
import pyemma
import deeptime as dt
import mdshare
import matplotlib.pyplot as plt

In [ ]:
plt.matplotlib.rcParams.update({'font.size': 16})

## well-sampled double-well potential

In [ ]:
file = mdshare.fetch('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    trjs = [fh['trajectory'][:, 1]]


In [ ]:
def implied_timescales_msm(dtrajs, lagtimes):
    """
    Estimate implied timescales based on the largest connected set of Markov states.
    
    Parameters
    ----------
    dtrajs: (list of) np.ndarray, dtype int 
        discrete trajectories or observations
    lagtimes: iterable of integers
        lag times for model evaluation
        
    Returns:
    -------
    deeptime.util.validation.ImpliedTimescales
        Object containing implied timescales of estimated MSMs
    """
    
    models = []
    for lag in lagtimes:
        count_model = dt.markov.TransitionCountEstimator(lag, 'sliding').fit_fetch(dtrajs)
        msm = dt.markov.msm.MaximumLikelihoodMSM().fit_fetch(count_model.submodel_largest())
        
        models.append(msm)
        
    return dt.util.validation.implied_timescales(models)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].hist(np.concatenate(trjs), bins=50, range=(-3, 3), alpha=1, density=True);
ax[0].plot(trjs[0][:200], np.linspace(0, 1, 200), color='k', alpha=.7)
ax[0].annotate('time', xy=(2.2, .8), xytext=(2.2, .1), rotation=90, 
            arrowprops=dict(arrowstyle='->'), ha='center',)


cl = dt.clustering.RegularSpace(dmin=0.05).fit(np.concatenate(trjs)).fetch_model()
dtrajs = [cl.transform(x) for x in trjs]
its = implied_timescales_msm(dtrajs, range(1, 11))

#its = implied_timescales_msm(dtrajs, lagtimes=range(1, 11))
dt.plots.plot_implied_timescales(its, marker='o', ax=ax[1], n_its=2)

fig.tight_layout()

In [ ]:
def plot_dw(trjs, hist=False, cl=None, ax=None, no_timeseries=False):
    """
    Plot double well as histogram / time-series overlay
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    if hist:
        ax.hist(trjs[0], bins=30, range=(-2, 2), alpha=.5, density=True);
        ax.hist(trjs[1], bins=30, range=(-2, 2), alpha=.5, density=True);
        
    if not no_timeseries:
        ax.annotate('time', xy=(1.8, .9), xytext=(1.8, .1), rotation=90, 
                    arrowprops=dict(arrowstyle='->'), ha='center',)
        if cl is None:
            ax.plot(trjs[0], np.linspace(0, 1, trjs[0].shape[0]), color='C0')
            ax.plot(trjs[1], np.linspace(0, 1, trjs[1].shape[0]), color='C1')
        else:
            dtrajs = [cl.transform(x) for x in trjs]
            ax.plot(cl.cluster_centers[dtrajs[0], 0], np.linspace(0, 1, dtrajs[0].shape[0]), color='C0')
            ax.plot(cl.cluster_centers[dtrajs[1], 0], np.linspace(0, 1, dtrajs[1].shape[0]), color='C1')
        
    ax.set_xlim(-2, 2)
    return ax

## different kinds of almost well-sampled double well potentials
### 1. irreversibly connected

In [ ]:
file = mdshare.fetch('doublewell_oneway.npy', working_directory='data')
trjs = [trj for trj in np.load(file)]

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
# cl = dt.clustering.RegularSpace(dmin=0.7).fit(np.concatenate(trjs)).fetch_model()
cl = dt.clustering.RegularSpace(dmin=0.1).fit(np.concatenate(trjs)).fetch_model()
print(cl.n_clusters)

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
dtrajs = [cl.transform(x) for x in trjs]
lagtimes = [1, 10, 100, 200, 300, 500, 800, 1000]

its = implied_timescales_msm(dtrajs, lagtimes)

ax = dt.plots.plot_implied_timescales(its, marker='o', n_its=2)
ax.semilogy()
ax.set_ylim(1e1, 3e5);

In [ ]:
c = dt.markov.TransitionCountEstimator(lagtimes[3], "sliding").fit(dtrajs).fetch_model()
m = dt.markov.msm.MaximumLikelihoodMSM().fit(c.submodel_largest()).fetch_model()

ax = plot_dw(trjs, hist=True, cl=cl)
ax.set_yticks([])
tx = ax.twinx()
tx.plot(cl.cluster_centers[m.count_model.state_symbols, 0], m.eigenvectors_right()[:, 1], 
        'ko:', label='first eigvec')
tx.set_ylabel('eigenvector')
ax.figure.legend()
ax.set_xlim(-2, 2)

In [ ]:
m.count_model.state_symbols

### 2. disconnected with crossovers

In [ ]:
file = mdshare.fetch('doublewell_disconnected.npy', working_directory='data')
trjs = [trj for trj in np.load(file)]

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
cl = dt.clustering.RegularSpace(dmin=0.7).fit(np.concatenate(trjs)).fetch_model()
# cl = dt.clustering.RegularSpace(dmin=0.1).fit(np.concatenate(trjs)).fetch_model()
print(cl.n_clusters)

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
dtrajs = [cl.transform(x) for x in trjs]
lagtimes = [1, 10, 100, 200, 300, 500, 800, 1000]
its = implied_timescales_msm(dtrajs, lagtimes)
ax = dt.plots.plot_implied_timescales(its, marker='o', n_its=2)
ax.semilogy()
ax.set_ylim(1e1, 3e5);

In [ ]:
c = dt.markov.TransitionCountEstimator(200, "sliding").fit(dtrajs).fetch_model()
m = dt.markov.msm.MaximumLikelihoodMSM().fit(c.submodel_largest()).fetch_model()
pcca = m.pcca(2)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
index_order = np.argsort(cl.cluster_centers[:, 0])
for n, metastable_distribution in enumerate(pcca.metastable_distributions):
    ax[0].step(cl.cluster_centers[index_order, 0], metastable_distribution[index_order], ':', linewidth=3,
               label='metastable distr state {}'.format(n), where='mid')
ax[0].set_title('metastable distributions')
for _ax in ax:
    tx = _ax.twinx()
    tx.set_yticklabels([]); tx.set_yticks([])
    plot_dw(trjs, cl=cl, hist=True, ax=tx, no_timeseries=True)
    
ax[1].step(cl.cluster_centers[index_order, 0], -np.log(m.stationary_distribution[index_order]), 'k--', linewidth=3,
           label='potential from MSM $\pi$', where='mid')
fig.legend(loc='center right')
ax[1].set_title('stat dist');

## some "real world data"

In [ ]:
import mdshare

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')
feat = pyemma.coordinates.featurizer(pdb)

feat.add_all()
data = pyemma.coordinates.load(files, features=feat)

### ill-conducted TICA analysis

In [ ]:
tica = dt.decomposition.TICA(lagtime=data[0].shape[0]-100, dim=2).fit(data).fetch_model()
tica_output = [tica.transform(x) for x in data]

pyemma.plots.plot_free_energy(*np.concatenate(tica_output).T, legacy=False);

### discretization and MSM estimation

In [ ]:
cluster = dt.clustering.KMeans(200, max_iter=50).fit(np.concatenate(tica_output)[::100]).fetch_model()

In [ ]:
lagtimes = [1, 5, 10, 20, 30, 50]
dtrajs = [cluster.transform(x) for x in tica_output]
its = implied_timescales_msm(dtrajs, lagtimes)
ax = dt.plots.plot_implied_timescales(its, marker='o', n_its=3);
ax.semilogy()

##### -> "converged"
### coarse graining into two states

In [ ]:
counts = dt.markov.TransitionCountEstimator(20, 'sliding').fit(dtrajs).fetch_model().submodel_largest()
msm_estimator = dt.markov.msm.MaximumLikelihoodMSM()
msm = msm_estimator.fit(counts).fetch_model()

nstates = 2
pcca = msm.pcca(nstates);

In [ ]:
stride = 10
metastable_trajs_strided = [pcca.assignments[dtraj[::stride]] for dtraj in dtrajs]
tica_output_strided = [y[::stride] for y in tica_output]
_, _, misc = pyemma.plots.plot_state_map(*np.concatenate(tica_output_strided).T, 
                                         np.concatenate(metastable_trajs_strided));
misc['cbar'].set_ticklabels(range(1, nstates + 1))  # set state numbers 1 ... nstates

In [ ]:
ck_lagtimes = np.arange(1, 11) * msm.lagtime
test_models = [dt.markov.msm.MaximumLikelihoodMSM(lagtime=lag).fit_fetch(dtrajs) for lag in ck_lagtimes]

In [ ]:
ck_test = msm.ck_test(test_models, nstates)
dt.plots.plot_ck_test(ck_test)

# 👍 AWESOME !


What could be wrong?


### let's have a look at the trajectories as assigned to PCCA coarse states

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 6), sharey=True, sharex=True)
ax_yticks_labels = []
for n, pcca_traj in enumerate(metastable_trajs_strided):
    ax.plot(range(len(pcca_traj)), pcca.n_metastable * n + pcca_traj, color='k', linewidth=0.3)
    ax.scatter(range(len(pcca_traj)), pcca.n_metastable * n + pcca_traj, c=pcca_traj, s=0.1)
    ax_yticks_labels.append(((pcca.n_metastable * (2 * n + 1) - 1) / 2, n + 1))
ax.set_yticks([l[0] for l in ax_yticks_labels])
ax.set_yticklabels([str(l[1]) for l in ax_yticks_labels])
ax.set_ylabel('Trajectory #')
ax.set_xlabel('time / {} ps'.format(stride))
fig.tight_layout()

### histogram view doesn't show connectedness nor metastability
### Take-away: Stay as close as possible to the trajectories

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6), sharex=True, sharey='row')

for n, trj in enumerate(tica_output):
    for dim, traj1d in enumerate(trj.T):
        axes[dim, n].plot(traj1d[::stride], linewidth=.5)
for ax in axes[1]:
    ax.set_xlabel('time / {} ps'.format(stride))
for dim, ax in enumerate(axes[:, 0]):
    ax.set_ylabel('IC {}'.format(dim + 1))
for n, ax in enumerate(axes[0]):
    ax.set_title('Trajectory # {}'.format(n + 1))
fig.tight_layout()

## Thanks for your attention.

An extended version of this notebook with more details & explanations can be found in the our PyEMMA tutorials

> Wehmeyer, C.; Scherer, M. K.; Hempel, T.; Husic, B. E.; Olsson, S.; Noé, F. Introduction to Markov State Modeling with the PyEMMA Software [Article v1.0]. LiveCoMS 2018, 1 (1), 5965. https://doi.org/10.33011/livecoms.1.1.5965.

or, more specifically, in 
[this notebook](
https://github.com/markovmodel/pyemma_tutorials/blob/master/notebooks/08-common-problems.ipynb).